In [ ]:
# Topology optimization code 
from __future__ import division
import numpy as np
from scipy.sparse import coo_matrix
from scipy  import linalg
from matplotlib import colors
import matplotlib.pyplot as plt
import sympy as sp
from sympy import *


## Task 1
In the following function, derive the stiffness matrix per element
$$\mathbf{K} = \int_{x,y} \mathbf{B}^\mathrm{T} \mathbf{C} \mathbf{B} \, dxdy$$
by first deriving the matrix for the shape function $\mathbf{B}$
Assumge plane stress condition and thickness is 1

In [ ]:
 def lk(varE,varnu):
    a, b, xi, eta, nu, E = symbols("a b x_1 x_2 nu E")
    # where a and b are the integration limits in this example -1 to 1
    # Type your code here
    # Need to define N_x for the four nodes of the element
    coordinates = Matrix([[0,0],[1,0],[1,1],[0,1]])
    t = 1
    E = varE
    nu = varnu

    Cc = (E/(1-nu**2)*Matrix([[1,nu,0],[nu,1,0],[0,0,(1-2*nu)/2]]))
    Shapefun = Matrix([[(1-xi)*(1-eta)],[(1+xi)*(1-eta)],
                   [(1+xi)*(1+eta)],[(1-xi)*(1+eta)]])/4
    Nn = Matrix([[0 for x in range(8)] for y in range(2)])

    for i in range(4):
        Nn[0,2*i] = Nn[1,2*i+1] = Shapefun[i]
        
    x = sum([Shapefun[i]*coordinates[i,0] for i in range(4)])
    y = sum([Shapefun[i]*coordinates[i,1] for i in range(4)])
    J = Matrix([[x.diff(xi), x.diff(eta)],[y.diff(xi),y.diff(eta)]])
    JinvT = J.inv().transpose()
    mat1 = Matrix([[1,0,0,0],[0,0,0,1],[0,1,1,0]])
    mat2 = Matrix([[JinvT[0,0],JinvT[0,1],0,0], 
                  [JinvT[1,0],JinvT[1,1],0,0], 
                  [0,0,JinvT[0,0],JinvT[0,1]], 
                  [0,0,JinvT[1,0],JinvT[1,1]]])
    mat3 = Matrix([[0 for x in range(8)] for y in range(4)])
    for i in range(4):
        mat3[0,2*i] = mat3[2,2*i+1] = sp.diff(Shapefun[i],xi)
        mat3[1,2*i] = mat3[3,2*i+1] = sp.diff(Shapefun[i],eta)
    B = mat1*mat2*mat3
    k1 = B.transpose()*Cc*B
    K = zeros(8)
    for i in range(8):
        for j in range(8):
            integrand=k1[i,j]*J.det()
            intlam=lambdify((eta,xi),integrand)
            #mp.dps is for integral accuracy (# of decimal points)
            # mp.dps = 8
            K[i,j] = sp.integrate(intlam,[-1,1],[-1,1])


    return K # why is it not working when i try to run this with other stuff???FWD:FGJkadskl'j;fghd aspkogiu k;hfadsgo;dfsahgop;'ifdsahg p[odi8afshgadops;ifghDS
    
    

## Task 2

In [ ]:
''' 
generate mapping matrix between local and global matrix
    input:
        nx - number of elements in the x direction
        ny - number of elements in the y direction
    output:
        edofMat - element dof matrix
'''
def edofMatFun (nx,ny):
    
    edofMat=np.zeros((nx*ny,8),dtype=int)


    # Type your code here
    for i in range(0,nx):
      for j in range(0,ny):
          el = (j+i*ny)
          edofMat[el][0] = 2*j +2*i*(ny+1)
          edofMat[el][1] = edofMat[el][0]+1
          edofMat[el][2] = edofMat[el][0]+2*(ny+1)
          edofMat[el][3] = edofMat[el][2]+1
          edofMat[el][4] = edofMat[el][3]+1
          edofMat[el][5] = edofMat[el][4]+1
          edofMat[el][6] = edofMat[el][1]+1
          edofMat[el][7] = edofMat[el][6]+1


    
    # Type above
    
    return edofMat

## Task 3

In [ ]:
''' solving for displacement based on updated stiffness
    a new global stiffness matrix is generated at every iteration
    by looping through each element in turn
    input:
        nx - number of elements in the x direction
        ny - number of elements in the y direction
        x  - current iteration optimization density
        penal - exponential penalty
        KE - element wise stiffness
    output:
        U - calculated displacement
'''
def FE(nx, ny, x, penal, KE, edofMat, freeDOF, fixedDOF): 
   
    ndof = 2*(nx+1)(ny+1)

    K = np.zeros([ndof, ndof])
    

    F = np.zeros([ndof,1])

    
    for i in np.arange(0,nx):
      for j in np.arange(0,ny):
        el = j+i*ny
        ix = np.ix_(edofMat[el],edofMat[el])
        K[K_id] = K[K_id]+x**penal*(Emax-Emin)*lk(Emax, nu)
        
    force = 2*(el+3+ny/2)+6
    F[force] = -1


    U[np.ix_(freeDOF)] = linalg.solve(K[np.ix_(freeDOF),np.ix_(freeDOF)],F[np.ix_(freeDOF)])
# why can i not figure this out?

    print(U)
    return U


## Task 4

In [ ]:
''' sensitivity filter
input:
nx - number of elements in the x direction
ny - number of elements in the y direction
x  - current iteration optimization density
dc - gradient of compliance
rmin - radius around the element
output:
dcn - updated gradient of compliance
'''
def sFilter(nx,ny,x,dc,rmin):
    dcn=np.zeros([ny,nx])
    #type 
    for i in np.arange(0,nx):
      for j in np.arange(0,ny):
        H_f = 0
        x_min = max(i- floor(rmin),0)
        x_max = min(i+ floor(rmin)+1,nx)
        y_min = max(j - floor(rmin),0)
        y_max = min(j+ floor(rmin)+1,ny)
        for k in np.arange(x_min,x_max):
          for l in np.arange(y_min,y_max):
            dist = rmin - sqrt((i-k)**2+(j-l)**2)
            H_f += max(0,dist)
            dcn[j,i] += max(0,dist)*x[j,i]*dc[j,i]
        dcn[j,i] = 1/(x[j,i]*H_f)*dcn[j,i]
       
    return dcn

## Task 5

In [ ]:
''' Optimality criteria
    input:
        nx - number of elements in the x direction
        ny - number of elements in the y direction
        x  - current iteration optimization density
        dc - gradient of compliance
        volfrac - volume fraction
        Emin - minimize youngs modulus
        Emax - maximum youngs modulus
    output:
        xnew - updated x
'''
def oc(nx,ny,x,dc,volfrac,Emin,Emax):
    l1 = 0
    l2 = 10e9
    m = 0.2
    l_m = .5*(l1+l2)
    
    while l2-l1 > 1e-4:
        x_n = x.copy()
        B_n = (-dc/(l_m))**.5
        
        for i in np.arange(0,nx):
          for j in np.arange(0,ny):
            if x[j,i]*B_n <= max(Emin,x[j,i]-m):
              x_n[j,i] = max(Emin,x[j,i]-m)
            elif x[j,i]*B_n >= min(Emax,x[j,i]+m):
              x_n[j,i] = min(Emax,x[j,i]+m)

            else:
              x_n[j,i] = x[j,i]*B_n[j,i]

        if sum(x_n)-volfrac*nx*ny >0 :
          l1 = l_m
        else:
          l2 = l_m
        
    
    return x_n

## Task 6

In [ ]:
# Also create the empty arrays in their appropriate sizing
# Possibly do this as/before you call the specific function that is necessary
nx=60
ny=20
volfrac=0.25
rmin=3
penal=3
nu   = 0.3 # Poisson's ratio
Emax = 1.0 # Maximum youngs modulus
Emin = 1e-4 # Minimum youngs modulus
ndof = 2*(nx+1)*(ny+1)
U = np.zeros([ndof,1])
fixedDOF = U[0:2*ny+1,0]
freeDOF = U[2*ny+1:,0]
F = np.zeros([ndof,1])


# def assemWinning(nx,ny,volfrac,rmin,penal,nu,Emax,Emin, freeDOF, fixedDOF)

x = volfrac*np.ones((ny,nx))
dc = np.zeros((ny,nx))





# Type your code to begin the bean counting
Ke = lk(Emax,nu)

eDofMat = edofMatFun (nx,ny)


# While change > 0.05 and counter < 10e6
#   c = 0 # initializing compliance
  
U = FE(nx, ny, x, penal, KE, edofMat) # This needs to be loop for the complaince to become smaller than 0.05 and some kind of max iteration counter to break the loop
 # Within or outside of this U array you still to create the c and dc arrays
#   for j in np.arange(0,ny):
#     for i in np.arange(0,nx):
#       edofMat[j,i]

#   dcn = sFilter(nx,ny,x,dc,rmin)

#   xnew = oc(nx,ny,x,dc,volfrac,Emin,Emax)
  
#   change = 


#   # Need to create change variable
# return change # xlist?



/usr/local/lib/python3.7/dist-packages/sympy/core/sympify.py:463: SymPyDeprecationWarning: 

String fallback in sympify has been deprecated since SymPy 1.6. Use
sympify(str(obj)) or sympy.core.sympify.converter or obj._sympy_
instead. See https://github.com/sympy/sympy/issues/18066 for more
info.

  deprecated_since_version='1.6'


SympifyError: ignored

In [ ]:
nx = 5
for i in np.arange(0,nx):
  print(i)
  print('\n')

0


1


2


3


4




## Task 7

In [ ]:
# Plot the density function
for x in xList:
    fig,ax = plt.subplots()
    ax.imshow(-x.reshape((ny,nx)),origin='lower', cmap='gray', interpolation='none',norm=colors.Normalize(vmin=-1,vmax=0))
plt.close()
'''
for i,x in enumerate(xList):
    fig,ax = plt.subplots()
    ax.imshow(-x,origin='lower', cmap='gray', interpolation='none',norm=colors.Normalize(vmin=-1,vmax=0))
plt.close()
'''